In [1]:
cd "/content/drive/My Drive/Generator Text by RNN"

/content/drive/My Drive/Generator Text by RNN


In [0]:
path_txt = "./1268-0.txt"

In [0]:
import numpy as np


In [0]:
with open("1268-0.txt", "r") as fp:
    text = fp.read()

In [0]:
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

In [0]:
text = text[start_indx:end_indx]

In [0]:
char_set = set(text) # Nhung ky tu xuat hien trong text

In [8]:
print('Total Length: ', len(text))
print('Unique Characters: ', len(char_set))

Total Length:  1112350
Unique Characters:  80


In [0]:
chars_sorted = sorted(char_set)

In [0]:
char2int = {ch:i for i, ch in enumerate(chars_sorted)}

In [0]:
char_array = np.array(chars_sorted) # decode from int to char

In [0]:
text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)

In [13]:
text_encoded.shape

(1112350,)

In [14]:
print(text[:15], '== Encoding ==>', text_encoded[:15])

THE MYSTERIOUS  == Encoding ==> [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]


In [15]:
print(text_encoded[15:21],'== Reverse ==>', ''.join(char_array[text_encoded[15:21]]) )

[33 43 36 25 38 28] == Reverse ==> ISLAND


In [0]:
import tensorflow as tf

ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

In [17]:
for ex in ds_text_encoded.take(5):
    print(ex.numpy(), '->', char_array[ex.numpy()])

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [0]:
seq_length = 40
chunk_size = seq_length +1

In [0]:
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

In [0]:
ds_sequences = ds_chunks.map(split_input_target)

In [22]:
for i in ds_sequences.take(2):
    print("Input: ",repr("".join(char_array[i[0].numpy()])))
    print("Target:",repr("".join(char_array[i[1].numpy()])))
    print('----')

Input:  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target: 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
----
Input:  ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
Target: 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'
----


In [23]:
c = 0
for i in ds_sequences:
    c+=1
print(c)

27130


In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.layers import Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = Sequential()

    model.add(Embedding(vocab_size, embedding_dim))
    model.add(LSTM(rnn_units, return_sequences=True))
    model.add(LSTM(rnn_units, return_sequences=True))

    model.add(Dense(vocab_size))
    return model
    

In [0]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

In [28]:
model = build_model(vocab_size=charset_size,
                    embedding_dim = embedding_dim,
                    rnn_units = rnn_units)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         20480     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1574912   
_________________________________________________________________
dense (Dense)                (None, None, 80)          41040     
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer="adam",
              loss = tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits = True
              ))

In [30]:
model.fit(ds, epochs = 20)

Epoch 1/20
424/424 [==============================] - 13s 32ms/step - loss: 2.3172
Epoch 2/20
424/424 [==============================] - 13s 31ms/step - loss: 1.7423
Epoch 3/20
424/424 [==============================] - 13s 31ms/step - loss: 1.5465
Epoch 4/20
424/424 [==============================] - 13s 31ms/step - loss: 1.4326
Epoch 5/20
424/424 [==============================] - 13s 32ms/step - loss: 1.3583
Epoch 6/20
424/424 [==============================] - 13s 31ms/step - loss: 1.3067
Epoch 7/20
424/424 [==============================] - 13s 31ms/step - loss: 1.2684
Epoch 8/20
424/424 [==============================] - 13s 31ms/step - loss: 1.2374
Epoch 9/20
424/424 [==============================] - 13s 31ms/step - loss: 1.2118
Epoch 10/20
424/424 [==============================] - 13s 31ms/step - loss: 1.1898
Epoch 11/20
424/424 [==============================] - 13s 31ms/step - loss: 1.1703
Epoch 12/20
424/424 [==============================] - 13s 31ms/step - loss: 1.1527
E

In [0]:
def sample(model, starting_str, 
           len_generated_text=500, 
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()    

        generated_str += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]

    return generated_str

In [0]:
tf.random.set_seed(1)

In [33]:
print(sample(model, starting_str='The island', scale_factor = 2.0))

The island was explored the depths of the forest, and a confident and potash through the colonists, who had been supplied a very prosperity for the forests of the sea.

The colonists appeared as if the brig can be less than a continent, the soul should be easily for the water’s boat which was not to explain the extremity of the island, straight for the first part of the island were supplied a regular being called his usual
completely deserted. The colonists were finished. The settlers were soon destroyed 
